### Imports

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import scipy
import cv2 as cv
import skimage

from src.models import InputImage
from src.fish import *
from src.well import find_well_props
from src.filters import sharpen_image

from skimage.feature import canny
from skimage.restoration import rolling_ball
from skimage.filters.ridges import hessian
from skimage.filters import frangi

### Functions

In [3]:
def is_fish(img: np.ndarray) -> bool:
    labeled = label(img)
    reg_props = regionprops(labeled)
    if reg_props[0].eccentricity > 0.9:  # 0.92
        fish = True
    else:
        fish = False
    return fish

In [4]:
def get_fish_convex_mask(input_img: InputImage) -> InputImage:
    for i in range(6):
        input_img = sharpen_image(input_img)
        input_img = sobel_edges(input_img)
        input_img = yen_thresholding(input_img)
        input_img = remove_meniscus(input_img)
        input_img = convex_hull_for_fish(input_img)
        if is_fish(input_img.binary):
            return input_img

    return input_img

In [ ]:
def refine_fish_convex_mask(input_img: InputImage) -> InputImage:
    bbox = input_img.fish_props.bounding_box
    remaining = np.zeros_like(input_img.fish_props.mask.masked)
    edges = sobel(input_img.fish_props.mask.masked[bbox.x1:bbox.x2, bbox.y1:bbox.y2])

    se = disk(2)
    x = binary_closing(binary_opening(yen_th(edges), se), se).astype(float)
    hull = convex_hull_image(x)

    remaining[bbox.x1:bbox.x2, bbox.y1:bbox.y2] = hull

    input_img.binary = remaining.copy()
    input_img.fish_props.mask.og = remaining.copy()
    input_img.fish_props.mask.masked = remaining * input_img.well_props.mask.cropped_masked
    input_img.fish_props.bounding_box = get_bounding_box(remaining)
    input_img.processed = remaining * input_img.well_props.mask.cropped_masked.copy()

    return input_img

# Refining fish mask

In [ ]:
fishy = InputImage("zf1.jpg")
fig, ax = plt.subplots(nrows=2, ncols=3, figsize=(25, 20))
ax = ax.ravel()
og = fishy.og.copy()
bg = rolling_ball(og)
og_bg = og - bg

fishy = find_well_props(fishy)

fishy = get_fish_convex_mask(fishy)
fishy = refine_fish_convex_mask(fishy)

ax[0].imshow(fishy.well_props.mask.cropped_masked, "binary")
ax[0].set_title('Well mask')
ax[1].imshow(fishy.fish_props.mask.masked, "binary")
ax[1].set_title('Fish masked')
ax[2].imshow(fishy.binary, "binary")
ax[2].set_title('Fish mask binary')
ax[3].imshow(bg, "binary")
ax[3].set_title('Fish mask binary')
ax[4].imshow(bg, "binary")
ax[4].set_title('Fish mask binary')

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=4, figsize=(20, 8))

bbox = fishy.fish_props.bounding_box
fishy.fish_props.mask.cropped = fishy.fininsh_props.mask.og[bbox.x1:bbox.x2, bbox.y1:bbox.y2]
fishy.fish_props.mask.cropped_masked = fishy.fish_props.mask.masked[bbox.x1:bbox.x2, bbox.y1:bbox.y2]

a = yen_th(sobel(fishy.fish_props.mask.cropped_masked))
cm = fishy.fish_props.mask.cropped_masked.copy()
bg = rolling_ball(cm)
eq_cm = equalize_hist(cm)

ax[0].imshow(cm, 'binary')
ax[1].imshow(bg, 'binary')
ax[2].imshow(yen_th(canny(cm)), 'binary')